In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats import linregress
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import numpy as np

DPI = 2000


In [2]:

# 温度数据
temperatures = np.array([75.79, 75.83, 75.83, 75.82])

# 计算平均值和方差
mean_temperature = np.mean(temperatures)
# 标准差
std_temperature = np.std(temperatures)


mean_temperature, std_temperature   



(75.8175, 0.01639359631075147)

In [3]:
mean_temperature += 273.15
mean_temperature

348.9675

In [4]:
import pandas as pd

# 数据
data = {
    "p (kPa)": [99.93, 95.30, 89.95, 84.97, 79.59, 74.96, 67.22, 64.15, 59.57, 57.04, 53.73, 50.23],
    "T (°C)": [75.82, 74.23, 72.38, 70.64, 68.63, 66.83, 63.67, 62.32, 60.18, 58.98, 57.29, 55.42]
}

# 转换为 DataFrame
df = pd.DataFrame(data)
df.head()

,p (kPa),T (°C)
0,99.93,75.82
1,95.30,74.23
2,89.95,72.38
3,84.97,70.64
4,79.59,68.63


In [5]:
# 添加 ln(p/p°) 列
p_standard = 100.00  # p° = 100.00 kPa
df["ln p/p°"] = np.log(df["p (kPa)"] / p_standard)

# 修改 T (°C) 列名称为 t (°C)
df.rename(columns={"T (°C)": "t (°C)"}, inplace=True)

# 添加 T (K) 列
df["T (K)"] = df["t (°C)"] + 273.15

# 添加 T^{-1} (K^{-1}) 列
df["T^{-1} (K^{-1})"] = 1 / df["T (K)"]

df.head()

,p (kPa),t (°C),ln p/p°,T (K),T^{-1} (K^{-1})
0,99.93,75.82,-0.000700,348.97,0.002866
1,95.30,74.23,-0.048140,347.38,0.002879
2,89.95,72.38,-0.105916,345.53,0.002894
3,84.97,70.64,-0.162872,343.79,0.002909
4,79.59,68.63,-0.228282,341.78,0.002926


In [6]:
# 交换 ln p/p° 与 t (°C) 两列的位置
df = df[['p (kPa)', 'ln p/p°', 't (°C)', 'T (K)', 'T^{-1} (K^{-1})']]

# 转换 T^{-1} (K^{-1}) 列为 T^{-1} (10^{-3} K^{-1})
df["T^{-1} (10^{-3} K^{-1})"] = df["T^{-1} (K^{-1})"] * 10**3

# 删除原始的 T^{-1} (K^{-1}) 列
df.drop(columns=["T^{-1} (K^{-1})"], inplace=True)

df.head()

,p (kPa),ln p/p°,t (°C),T (K),T^{-1} (10^{-3} K^{-1})
0,99.93,-0.000700,75.82,348.97,2.865576
1,95.30,-0.048140,74.23,347.38,2.878692
2,89.95,-0.105916,72.38,345.53,2.894105
3,84.97,-0.162872,70.64,343.79,2.908752
4,79.59,-0.228282,68.63,341.78,2.925859


In [7]:
-int(np.floor(np.log10(abs(0.228282)))) + 3

4

In [8]:
df["ln p/p°"] = df["ln p/p°"].apply(lambda x: round(x, 4) if x > 0 else round(x, -int(np.floor(np.log10(abs(x)))) + 3))

# 保留五位有效数字
df["T^{-1} (10^{-3} K^{-1})"] = df["T^{-1} (10^{-3} K^{-1})"].round(4)

df.head()

,p (kPa),ln p/p°,t (°C),T (K),T^{-1} (10^{-3} K^{-1})
0,99.93,-0.00070,75.82,348.97,2.8656
1,95.30,-0.04814,74.23,347.38,2.8787
2,89.95,-0.10590,72.38,345.53,2.8941
3,84.97,-0.16290,70.64,343.79,2.9088
4,79.59,-0.22830,68.63,341.78,2.9259


In [9]:
print(df.to_latex(index=False))

\begin{tabular}{rrrrr}
\toprule
p (kPa) & ln p/p° & t (°C) & T (K) & T^{-1} (10^{-3} K^{-1}) \\
\midrule
99.930000 & -0.000700 & 75.820000 & 348.970000 & 2.865600 \\
95.300000 & -0.048140 & 74.230000 & 347.380000 & 2.878700 \\
89.950000 & -0.105900 & 72.380000 & 345.530000 & 2.894100 \\
84.970000 & -0.162900 & 70.640000 & 343.790000 & 2.908800 \\
79.590000 & -0.228300 & 68.630000 & 341.780000 & 2.925900 \\
74.960000 & -0.288200 & 66.830000 & 339.980000 & 2.941300 \\
67.220000 & -0.397200 & 63.670000 & 336.820000 & 2.968900 \\
64.150000 & -0.443900 & 62.320000 & 335.470000 & 2.980900 \\
59.570000 & -0.518000 & 60.180000 & 333.330000 & 3.000000 \\
57.040000 & -0.561400 & 58.980000 & 332.130000 & 3.010900 \\
53.730000 & -0.621200 & 57.290000 & 330.440000 & 3.026300 \\
50.230000 & -0.688600 & 55.420000 & 328.570000 & 3.043500 \\
\bottomrule
\end{tabular}



In [10]:
def float_to_sci(f: float, digits=2):
    if f == 0:
        return '0.' + '0' * digits
    m, e = '%e' % f, int(np.floor(np.log10(abs(f))))
    # return r'...'
    if 0 <= abs(e) < 3:
        return f'{f:.{digits}f}'
    if f < 0:
        digits += 1
    return r'${}\times 10^{{{}}}$'.format(m[:digits + 2], e)


In [11]:
def format_y(value, tick_number):
    return f'{value:.2f}'

def format_x(value, tick_number):
    return f'{value:.2f}'

In [12]:

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

plt.figure(figsize=(8, 6), dpi=DPI)
# marker为小球：o，方块：s，星星：*，加号：+，叉叉：x，三角形：^，倒三角：v，菱形：D，六边形：h，五边形：p，八边形：8，圈圈：o，
plt.scatter(df['T (K)'], df['p (kPa)'], marker='o', color='black')
plt.plot(df['T (K)'], df['p (kPa)'], color='blue')

plt.xlabel(r'$T$ / $\mathrm{K}$')
plt.ylabel(r'$p$ / $\mathrm{kPa}$')
plt.legend(frameon=False)

plt.gca().xaxis.set_major_formatter(FuncFormatter(format_x))
plt.gca().yaxis.set_major_formatter(FuncFormatter(format_y))
plt.gca().xaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.gca().yaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.grid(which='major', alpha=0.5)
plt.grid(which='minor', alpha=0.2)

plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [13]:
# 拟合函数df['T^{-1} (10^{-3} K^{-1})'], df['ln p/p°']
def func(x, a, b):
    return a * x + b

# 拟合
popt, pcov = curve_fit(func, df['T^{-1} (10^{-3} K^{-1})'], df['ln p/p°'])

# 拟合参数
a = popt[0]
b = popt[1]
# 求参数误差
a_err = np.sqrt(pcov[0][0])
b_err = np.sqrt(pcov[1][1])

# 拟合曲线
x = np.linspace(df['T^{-1} (10^{-3} K^{-1})'].min(), df['T^{-1} (10^{-3} K^{-1})'].max(), 1000)
y = func(x, a, b)

In [21]:
# 打印拟合参数
r2 = linregress(df['T^{-1} (10^{-3} K^{-1})'], df['ln p/p°']).rvalue ** 2
print('a =', a, '+-', a_err)
print('b =', b, '+-', b_err)
print('R^2 =', r2)

a = -3.882439800837582 +- 0.009435270087755245
b = 11.129020858059002 +- 0.027874620945108633
R^2 = 0.9999409427041349


In [20]:
def format_y(value, tick_number):
    return f'{value:.2f}'

def format_x(value, tick_number):
    return f'{value:.3f}'

plt.figure(figsize=(8, 6), dpi=DPI)

plt.scatter(df['T^{-1} (10^{-3} K^{-1})'], df['ln p/p°'], marker='o', color='black')
plt.plot(x, y, color='red', label= 'Linear fit')

plt.xlabel(r'$T^{-1}\times 10^{3}$ / $\mathrm{K^{{-1}}}$')
plt.ylabel(r'$\ln\dfrac{p}{p^\ominus}$')
plt.legend(frameon=False)

plt.gca().xaxis.set_major_formatter(FuncFormatter(format_x))
plt.gca().yaxis.set_major_formatter(FuncFormatter(format_y))
plt.gca().xaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.gca().yaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.grid(which='major', alpha=0.5)
plt.grid(which='minor', alpha=0.2)

plt.show()

In [16]:
ln_ps_p0 = np.log(101.325 / 100.00)

T_ps = a / (ln_ps_p0-b) * 10**3
T_ps

349.2703708258414

In [17]:
import sympy as sp
import uncertainties as uc
import uncertainties.unumpy as unp
import math
# 计算误差并打印过程
a_un = uc.ufloat(a, a_err)
b_un = uc.ufloat(b, b_err)

T_ps_un = a_un / (ln_ps_p0-b_un) * 10**3
T_ps_un

349.2703708258414+/-1.219666932274597

In [18]:
d_H_unc = -a_un * 8.314
d_H_unc

32.27860450416365+/-0.0784448355095971

In [19]:
d_S_unc1 = b_un * 8.314
d_S_unc2 = d_H_unc / T_ps_un * 10**3
d_S_unc1, d_S_unc2

(92.52667941390254+/-0.2317495985376332,
 92.41724234392304+/-0.2317495985376332)